In [5]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
from numpy import sin, cos
import numba as nb
import numdifftools as nd

In [2]:
# power flow function (equality constraints)
def gfun(x, u, p):
  
    VM3 = x[0]
    VA3 = x[1]
    VA2 = x[2]
    
    VM1 = u[0]
    P2 = u[1]
    VM2 = u[2]
    
    VA1 = p[0]
    P3 = p[1]
    Q3 = p[2]

    # intermediate quantities
    VA23 = VA2 - VA3
    VA31 = VA3 - VA1
    VA32 = VA3 - VA2
    
    F1 = 4.0*VM2*VM2 + VM2*VM3*(-4*cos(VA23) + 10*sin(VA23)) - P2
    F2 = (8.0*VM3*VM3 + VM3*VM1*(-4*cos(VA31) + 5*sin(VA31))
          + VM3*VM2*(-4*cos(VA32) + 10*sin(VA32)) + P3)
    F3 = (15.0*VM3*VM3 + VM3*VM1*(-4*sin(VA31) - 5*cos(VA31))
          + VM3*VM2*(-4*sin(VA32) - 10*cos(VA32)) + Q3)

    return np.array([F1, F2, F3])

# cost function
def cfun(x, u, p):

    VM3 = x[0]
    VA3 = x[1]

    VM1 = u[0]
    P2 = u[1]

    VA1 = p[0]

    VA13 = VA1 - VA3
    
    w1 = 1.0
    w2 = 1.0

    cost = (w1*(4.0*VM1*VM1 + VM1*VM3*(-4*cos(VA13) + 5*sin(VA13))) +
          w2*P2)
    
    return cost

Initialize script with same initial conditions as in the paper

In [3]:
# initial parameters
x = np.zeros(3)
u = np.zeros(3)
p = np.zeros(3)

# this is an initial guess
x[0] = 1.0 #VM3
x[1] = 0.0 #VA3
x[2] = 0.0 #VA2

# this is given by the problem data, but might be "controlled" via OPF
u[0] = 1.0 #VM1
u[1] = 1.7 #P2
u[2] = 1.0 #VM2

# these parameters are fixed through the computation
p[0] = 0.0 #VA1, slack angle
p[1] = 2.0 #P3
p[2] = 1.0 #Q3

# print initial guesses
print(x)
print(u)

[1. 0. 0.]
[1.  1.7 1. ]


In [8]:
jac = nd.Gradient(cfun, 0)
print(jac(x, u, p))

IndexError: list index out of range